## **Mount driver**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Import lib**

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA

import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.utils import compute_class_weight
import tensorflow as tf
import random as python_random

from sklearn import metrics
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

## **Import dataset**

In [3]:
LM_dataset = pd.read_csv("/content/drive/MyDrive/LabRISE/CodeSmellDetection/embedding-dataset/software-metrics/LongMethod_code_metrics_values.csv")
FE_dataset = pd.read_csv("/content/drive/MyDrive/LabRISE/CodeSmellDetection/embedding-dataset/software-metrics/FeatureEnvy_code_metrics_values.csv")
GC_dataset = pd.read_csv("/content/drive/MyDrive/LabRISE/CodeSmellDetection/embedding-dataset/software-metrics/GodClass_code_metrics_values.csv")
DC_dataset = pd.read_csv("/content/drive/MyDrive/LabRISE/CodeSmellDetection/embedding-dataset/software-metrics/GodClass_code_metrics_values.csv")

In [4]:
print(LM_dataset.isna().sum())

label                             0
sample_id                         0
constructor                       0
cbo                               0
wmc                               0
rfc                               0
loc                               0
returnsQty                        0
variablesQty                      0
parametersQty                     0
methodsInvokedQty                 0
methodsInvokedLocalQty            0
methodsInvokedIndirectLocalQty    0
loopQty                           0
comparisonsQty                    0
tryCatchQty                       0
parenthesizedExpsQty              0
stringLiteralsQty                 0
numbersQty                        0
assignmentsQty                    0
mathOperationsQty                 0
maxNestedBlocksQty                0
anonymousClassesQty               0
lambdasQty                        0
uniqueWordsQty                    0
modifiers                         0
logStatementsQty                  0
hasJavaDoc                  

## **Data Processing**

- tcc' and 'lcc' in the GC_dataset and DC_dataset have a lot of NaN values.

In [5]:
GC_dataset = GC_dataset.drop(['tcc', 'lcc'], axis=1)
DC_dataset = DC_dataset.drop(['tcc', 'lcc'], axis=1)

In [7]:
X_LM, y_LM = LM_dataset.drop(['label', 'sample_id'], axis=1).values, LM_dataset['label'].values
X_FE, y_FE = FE_dataset.drop(['label', 'sample_id'], axis=1).values, FE_dataset['label'].values
X_GC, y_GC = GC_dataset.drop(['label', 'sample_id'], axis=1).values, GC_dataset['label'].values
X_DC, y_DC = DC_dataset.drop(['label', 'sample_id'], axis=1).values, DC_dataset['label'].values

##**K-fold Cross Validation model evaluation**

In [8]:
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import KernelPCA

import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.utils import compute_class_weight
import tensorflow as tf
import random as python_random

from sklearn import metrics
from sklearn.metrics import confusion_matrix

from keras import backend as K

In [9]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, f1_score, matthews_corrcoef, roc_curve, auc

In [10]:
def evaluation_metrics(y_test, y_pred):
    # Performance Measures
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    # precision
    precision = tp / (tp + fp)

    # recall
    recall = tp / (tp + fn)

    # Calculate performance measures for the fold
    fscore = f1_score(y_test, y_pred)

    # mcc
    mcc = matthews_corrcoef(y_test, y_pred)

    return precision, recall, fscore, mcc

# **Naive Bayes**

In [ ]:
def train_test_KFold(X, y, feature_scaling=StandardScaler()):
    print("============= RUNNING NAIVE BAYES =============")
    kfold = StratifiedKFold(n_splits=5, shuffle=True)

    # Define a range of var_smoothing values to try
    var_smoothing_values = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]

    best_performance = [0,0,0,0]

    # Iterate over var_smoothing values and choose the best one
    for var_smoothing in var_smoothing_values:
        fold_no = 1
        precision_per_fold = []
        recall_per_fold = []
        fscore_per_fold = []
        mcc_per_fold = []

        classifier = GaussianNB(var_smoothing=var_smoothing)
        for train, test in kfold.split(X,y):
            # print(f'Training for fold {fold_no}: ')
            X_train = X[train]
            X_test = X[test]
            y_train = y[train]
            y_test = y[test]
            index, counts = np.unique(y_train, return_counts=True)
            print(f'{index}, {counts}')

            # Feature Scaling
            if feature_scaling != None:
                X_train = feature_scaling.fit_transform(X_train)
                X_test = feature_scaling.transform(X_test)

            # Define the model, fit and predict
            classifier.fit(X_train, y_train)
            y_pred = classifier.predict(X_test)

            precision, recall, fscore, mcc = evaluation_metrics(y_test, y_pred)

            precision_per_fold.append(precision)
            recall_per_fold.append(recall)
            fscore_per_fold.append(fscore)
            mcc_per_fold.append(mcc)

            # Increase fold number
            fold_no = fold_no + 1

        precision_kfold = sum(precision_per_fold)/len(precision_per_fold)
        recall_kfold = sum(recall_per_fold)/len(recall_per_fold)
        fscore_kfold = sum(fscore_per_fold)/len(fscore_per_fold)
        mcc_kfold = sum(mcc_per_fold)/len(mcc_per_fold)

        print(f"VAR SMOORING = {var_smoothing} ==> {precision_kfold}, {recall_kfold}, {fscore_kfold}, {mcc_kfold}")
        if fscore_kfold > best_performance[2]:
            best_performance[0] = precision_kfold
            best_performance[1] = recall_kfold
            best_performance[2] = fscore_kfold
            best_performance[3] = mcc_kfold

    return best_performance

In [ ]:
# Long Method
print(train_test_KFold(X_LM, y_LM))

============= RUNNING NAIVE BAYES =============
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
VAR SMOORING = 1e-09 ==> 0.5599122493183004, 0.829935064935065, 0.6682585743960092, 0.6320368914124039
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
VAR SMOORING = 1e-08 ==> 0.5650250649613474, 0.8303896103896105, 0.6694465656711974, 0.6343017671218214
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
VAR SMOORING = 1e-07 ==> 0.5550740727608436, 0.8231818181818182, 0.6625471334698385, 0.6250763177198929
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
VAR SMOORING = 1e-06 ==> 0.5533340818928995, 0.8196103896103896, 0.6602373864681677, 0.6222783558060294
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
VAR SMOORING = 1e-05 ==> 0.5697794330029479, 0.819610389610389

In [ ]:
# Feature Envy
print(train_test_KFold(X_FE, y_FE))

============= RUNNING NAIVE BAYES =============
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
VAR SMOORING = 1e-09 ==> 0.11458429072720937, 0.7406593406593408, 0.1981490319718942, 0.2431188927361716
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
VAR SMOORING = 1e-08 ==> 0.11620254106574215, 0.743956043956044, 0.1993106771676197, 0.2453442372238311
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
VAR SMOORING = 1e-07 ==> 0.10464071250990356, 0.6956043956043956, 0.18174150556574603, 0.21942726550732433
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
VAR SMOORING = 1e-06 ==> 0.0943000071533109, 0.7274725274725274, 0.1641818628162839, 0.1906489815862609
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
VAR SMOORING = 1e-05 ==> 0.11442678998879288, 0.739560439

In [ ]:
# God Class
print(train_test_KFold(X_GC, y_GC))

============= RUNNING NAIVE BAYES =============
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-09 ==> 0.15778537231816187, 0.8342731829573935, 0.26523735329137954, 0.11879935275756404
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-08 ==> 0.16197588658302772, 0.8308270676691729, 0.2709095918746897, 0.12798057532301738
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-07 ==> 0.1609591938924435, 0.8416666666666668, 0.2702034836008014, 0.1299964555495441
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-06 ==> 0.15381859457551655, 0.8095864661654135, 0.2584497576313391, 0.09809558099208499
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-05 ==> 0.1615265799408659, 0.8382205

In [ ]:
# Data Class
print(train_test_KFold(X_DC, y_DC))

============= RUNNING NAIVE BAYES =============
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-09 ==> 0.16438306501844818, 0.8204887218045112, 0.27338764185280473, 0.12908958211860022
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-08 ==> 0.16043205692888907, 0.8414160401002506, 0.26942609749878604, 0.12577197098489948
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-07 ==> 0.15567619590578372, 0.8204887218045112, 0.26168326201045133, 0.10722038761990552
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-06 ==> 0.15798974697214324, 0.8271929824561404, 0.26524849745626977, 0.11700500767087871
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
VAR SMOORING = 1e-05 ==> 0.15652494096505074, 0.8

# **Nearest neighbor**

In [ ]:
def train_test_KFold(X, y, feature_scaling=StandardScaler()):
    print("============= RUNNING NEAREST NEIGHBOR =============")
    kfold = StratifiedKFold(n_splits=5, shuffle=True)

    #  Define hyperparameters to search over
    k_values = [1, 3, 5, 7, 9]  # Example values for the number of neighbors (k)

    best_performance = [0,0,0,0]

    # Iterate over var_smoothing values and choose the best one
    for k in k_values:
        fold_no = 1
        precision_per_fold = []
        recall_per_fold = []
        fscore_per_fold = []
        mcc_per_fold = []

        classifier = KNeighborsClassifier(n_neighbors=k)
        for train, test in kfold.split(X,y):
            # print(f'Training for fold {fold_no}: ')
            X_train = X[train]
            X_test = X[test]
            y_train = y[train]
            y_test = y[test]

            # Feature Scaling
            if feature_scaling != None:
                X_train = feature_scaling.fit_transform(X_train)
                X_test = feature_scaling.transform(X_test)

            # Define the model, fit and predict
            classifier.fit(X_train, y_train)
            y_pred = classifier.predict(X_test)

            precision, recall, fscore, mcc = evaluation_metrics(y_test, y_pred)

            precision_per_fold.append(precision)
            recall_per_fold.append(recall)
            fscore_per_fold.append(fscore)
            mcc_per_fold.append(mcc)

            # Increase fold number
            fold_no = fold_no + 1

        precision_kfold = sum(precision_per_fold)/len(precision_per_fold)
        recall_kfold = sum(recall_per_fold)/len(recall_per_fold)
        fscore_kfold = sum(fscore_per_fold)/len(fscore_per_fold)
        mcc_kfold = sum(mcc_per_fold)/len(mcc_per_fold)

        print(f"K-Nearest Neighbor = {k} ==> {precision_kfold}, {recall_kfold}, {fscore_kfold}, {mcc_kfold}")
        if fscore_kfold > best_performance[2]:
            best_performance[0] = precision_kfold
            best_performance[1] = recall_kfold
            best_performance[2] = fscore_kfold
            best_performance[3] = mcc_kfold

    return best_performance

In [ ]:
# Long Method
print(train_test_KFold(X_LM, y_LM))

============= RUNNING NEAREST NEIGHBOR =============
K-Nearest Neighbor = 1 ==> 0.6942517006802721, 0.5488311688311688, 0.6118714368338428, 0.5737275071303852
K-Nearest Neighbor = 3 ==> 0.804768962921137, 0.531038961038961, 0.635964163426652, 0.6172803308037386
K-Nearest Neighbor = 5 ==> 0.8367388167388168, 0.5128571428571428, 0.6357548617587698, 0.6228067621886344
K-Nearest Neighbor = 7 ==> 0.8834129311548666, 0.4943506493506494, 0.6335261283687448, 0.6315116813587467
K-Nearest Neighbor = 9 ==> 0.9051698301698302, 0.4798701298701299, 0.626286844499019, 0.630636266537744
[0.804768962921137, 0.531038961038961, 0.635964163426652, 0.6172803308037386]


In [ ]:
# Feature Envy
print(train_test_KFold(X_FE, y_FE))

============= RUNNING NEAREST NEIGHBOR =============
K-Nearest Neighbor = 1 ==> 0.12653846153846154, 0.12197802197802199, 0.12257742257742257, 0.09866990125195277
K-Nearest Neighbor = 3 ==> 0.24, 0.05934065934065934, 0.09333333333333335, 0.10522080149647986
K-Nearest Neighbor = 5 ==> 0.16666666666666666, 0.04395604395604395, 0.06666666666666668, 0.07178064345798578


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


K-Nearest Neighbor = 7 ==> nan, 0.015384615384615385, 0.025, 0.025824294453517214
K-Nearest Neighbor = 9 ==> nan, 0.046153846153846156, 0.07301587301587302, 0.10168204920943176
[0.12653846153846154, 0.12197802197802199, 0.12257742257742257, 0.09866990125195277]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


In [ ]:
# God Class
print(train_test_KFold(X_GC, y_GC))

============= RUNNING NEAREST NEIGHBOR =============
K-Nearest Neighbor = 1 ==> 0.4446330042223979, 0.46127819548872184, 0.4508246667868471, 0.36703775524391313
K-Nearest Neighbor = 3 ==> 0.5415490912155794, 0.39429824561403504, 0.4552294371262559, 0.39435676265113634
K-Nearest Neighbor = 5 ==> 0.5771317007956176, 0.37305764411027564, 0.45153219712283965, 0.401175664615116
K-Nearest Neighbor = 7 ==> 0.6054910394265234, 0.3343984962406015, 0.4291615586823535, 0.39135744021326324
K-Nearest Neighbor = 9 ==> 0.6566739926739926, 0.32738095238095244, 0.43498846782570777, 0.41136578808906316
[0.5415490912155794, 0.39429824561403504, 0.4552294371262559, 0.39435676265113634]


In [ ]:
# Data Class
print(train_test_KFold(X_DC, y_DC))

============= RUNNING NEAREST NEIGHBOR =============
K-Nearest Neighbor = 1 ==> 0.4567139365036863, 0.47525062656641603, 0.4646486208243508, 0.3821083462227478
K-Nearest Neighbor = 3 ==> 0.5114926140127312, 0.37706766917293233, 0.4327603284281101, 0.3679992858880021
K-Nearest Neighbor = 5 ==> 0.616175878953732, 0.39423558897243105, 0.47733585729065364, 0.4332951660527284
K-Nearest Neighbor = 7 ==> 0.6160876757650952, 0.3382205513784461, 0.43372086276887145, 0.3979206860627848
K-Nearest Neighbor = 9 ==> 0.6262930639943186, 0.3169172932330827, 0.41749799047816205, 0.3874160966378853
[0.616175878953732, 0.39423558897243105, 0.47733585729065364, 0.4332951660527284]


# **Random Forest**

In [ ]:
def train_test_KFold(X, y, feature_scaling=StandardScaler()):
    print("============= RUNNING RANDOM FOREST =============")
    kfold = StratifiedKFold(n_splits=5, shuffle=True)

    #  Define hyperparameters to search over
    n_estimators_values = [10, 50, 100, 200]  # Example values for the number of trees
    max_depth_values = [None, 10, 20, 30]  # Example values for the maximum depth of trees

    best_performance = [0,0,0,0]

    # Iterate over var_smoothing values and choose the best one
    for n_estimators in n_estimators_values:
        for max_depth in max_depth_values:
            fold_no = 1
            precision_per_fold = []
            recall_per_fold = []
            fscore_per_fold = []
            mcc_per_fold = []

            classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            for train, test in kfold.split(X,y):
                # print(f'Training for fold {fold_no}: ')
                X_train = X[train]
                X_test = X[test]
                y_train = y[train]
                y_test = y[test]

                # Feature Scaling
                if feature_scaling != None:
                    X_train = feature_scaling.fit_transform(X_train)
                    X_test = feature_scaling.transform(X_test)

                # Define the model, fit and predict
                classifier.fit(X_train, y_train)
                y_pred = classifier.predict(X_test)

                precision, recall, fscore, mcc = evaluation_metrics(y_test, y_pred)

                precision_per_fold.append(precision)
                recall_per_fold.append(recall)
                fscore_per_fold.append(fscore)
                mcc_per_fold.append(mcc)

                # Increase fold number
                fold_no = fold_no + 1

            precision_kfold = sum(precision_per_fold)/len(precision_per_fold)
            recall_kfold = sum(recall_per_fold)/len(recall_per_fold)
            fscore_kfold = sum(fscore_per_fold)/len(fscore_per_fold)
            mcc_kfold = sum(mcc_per_fold)/len(mcc_per_fold)

            print(f"Trees = {n_estimators} - max depth = {max_depth} ==> {precision_kfold}, {recall_kfold}, {fscore_kfold}, {mcc_kfold}")
            if fscore_kfold > best_performance[2]:
                best_performance[0] = precision_kfold
                best_performance[1] = recall_kfold
                best_performance[2] = fscore_kfold
                best_performance[3] = mcc_kfold

    return best_performance

In [ ]:
# Long Method
print(train_test_KFold(X_LM, y_LM))

============= RUNNING RANDOM FOREST =============
Trees = 10 - max depth = None ==> 0.7716903255266427, 0.6138961038961039, 0.6818310924214759, 0.6519569419248177
Trees = 10 - max depth = 10 ==> 0.7932012089415673, 0.6896753246753247, 0.734490145323984, 0.7067775532070881
Trees = 10 - max depth = 20 ==> 0.7599878419452888, 0.6314285714285715, 0.6887083578811801, 0.6565338850325723
Trees = 10 - max depth = 30 ==> 0.791032862508359, 0.638961038961039, 0.7021409039632545, 0.675725365414599
Trees = 50 - max depth = None ==> 0.8137289570619604, 0.6820779220779221, 0.7416017313246701, 0.7149706922836321
Trees = 50 - max depth = 10 ==> 0.777644333891361, 0.6751948051948051, 0.7217705567949639, 0.6912629322600787
Trees = 50 - max depth = 20 ==> 0.7943363499245851, 0.6818831168831169, 0.7320937858665355, 0.7041030620368388
Trees = 50 - max depth = 30 ==> 0.7967507533099507, 0.678961038961039, 0.7323956938086751, 0.704077916133361
Trees = 100 - max depth = None ==> 0.7999424237780218, 0.68246753

In [ ]:
# Feature Envy
print(train_test_KFold(X_FE, y_FE))

============= RUNNING RANDOM FOREST =============


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Trees = 10 - max depth = None ==> nan, 0.02967032967032967, 0.05166666666666666, 0.08081058457199577
Trees = 10 - max depth = 10 ==> nan, 0.015384615384615385, 0.028571428571428574, 0.04905143443043736


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Trees = 10 - max depth = 20 ==> nan, 0.015384615384615385, 0.028571428571428574, 0.05240510448559179
Trees = 10 - max depth = 30 ==> 0.3, 0.046153846153846156, 0.08000000000000002, 0.1083832254607705


...

Trees = 200 - max depth = 10 ==> nan, 0.0, 0.0, 0.0


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Trees = 200 - max depth = 20 ==> nan, 0.0, 0.0, -0.003270644018227125


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Trees = 200 - max depth = 30 ==> nan, 0.015384615384615385, 0.026666666666666672, 0.03526358401797551
[0.3, 0.046153846153846156, 0.08000000000000002, 0.1083832254607705]


In [ ]:
# God Class
print(train_test_KFold(X_GC, y_GC))

============= RUNNING RANDOM FOREST =============
Trees = 10 - max depth = None ==> 0.6703979938818649, 0.3419172932330827, 0.4510326337390692, 0.4270075520950936
Trees = 10 - max depth = 10 ==> 0.6706148305717271, 0.3487468671679198, 0.4579485829227095, 0.4320365632522787
Trees = 10 - max depth = 20 ==> 0.7045995256340084, 0.33101503759398493, 0.4487209552873027, 0.4336500140783409
Trees = 10 - max depth = 30 ==> 0.6755629255629255, 0.3454887218045113, 0.4518593417037121, 0.4298518587628912
Trees = 50 - max depth = None ==> 0.7441554520358868, 0.3662280701754386, 0.4895147212490012, 0.4761291820576797
Trees = 50 - max depth = 10 ==> 0.736815829897324, 0.32387218045112787, 0.44613360323886636, 0.44121149946849736
Trees = 50 - max depth = 20 ==> 0.705897120121258, 0.3556390977443609, 0.4725362029612611, 0.45255301761519295
Trees = 50 - max depth = 30 ==> 0.7331556948798328, 0.34154135338345865, 0.46324521492715204, 0.4523497539803816
Trees = 100 - max depth = None ==> 0.7230885780885781

In [ ]:
# Data Class
print(train_test_KFold(X_DC, y_DC))

============= RUNNING RANDOM FOREST =============
Trees = 10 - max depth = None ==> 0.6730630630630631, 0.3593984962406015, 0.46147043763742046, 0.4359169105477395
Trees = 10 - max depth = 10 ==> 0.6848925970640156, 0.3978696741854636, 0.5018937813574527, 0.47032484308018024
Trees = 10 - max depth = 20 ==> 0.7043727598566308, 0.3276315789473684, 0.4462957771340165, 0.4322488534725887
Trees = 10 - max depth = 30 ==> 0.733157773661206, 0.3382205513784461, 0.4598243626250027, 0.4513375986632505
Trees = 50 - max depth = None ==> 0.7476967285587977, 0.36641604010025064, 0.4909482952451941, 0.4781619991524426
Trees = 50 - max depth = 10 ==> 0.7532317145220372, 0.33433583959899743, 0.46057135635448887, 0.45638419503697786
Trees = 50 - max depth = 20 ==> 0.7315727164114261, 0.36641604010025064, 0.48367596525246126, 0.46875216841928297
Trees = 50 - max depth = 30 ==> 0.734292450844175, 0.3768170426065163, 0.4905444893832153, 0.4741146971047945
Trees = 100 - max depth = None ==> 0.74156173261436

# **Logistic Regression**

In [ ]:
def train_test_KFold(X, y, feature_scaling=StandardScaler()):
    print("============= RUNNING LOGISTIC REGRESSION =============")
    kfold = StratifiedKFold(n_splits=5, shuffle=True)

    # Define hyperparameters to search over
    C_values = [0.001, 0.01, 0.1, 1, 10, 100]  # Example values for the regularization strength (C)

    best_performance = [0,0,0,0]

    # Iterate over var_smoothing values and choose the best one
    for C in C_values:
        fold_no = 1
        precision_per_fold = []
        recall_per_fold = []
        fscore_per_fold = []
        mcc_per_fold = []

        classifier = LogisticRegression(C=C, random_state=42)
        for train, test in kfold.split(X,y):
            # print(f'Training for fold {fold_no}: ')
            X_train = X[train]
            X_test = X[test]
            y_train = y[train]
            y_test = y[test]
            index, counts = np.unique(y_train, return_counts=True)
            print(f'{index}, {counts}')

            # Feature Scaling
            if feature_scaling != None:
                X_train = feature_scaling.fit_transform(X_train)
                X_test = feature_scaling.transform(X_test)

            # Define the model, fit and predict
            classifier.fit(X_train, y_train)
            y_pred = classifier.predict(X_test)

            precision, recall, fscore, mcc = evaluation_metrics(y_test, y_pred)

            precision_per_fold.append(precision)
            recall_per_fold.append(recall)
            fscore_per_fold.append(fscore)
            mcc_per_fold.append(mcc)

            # Increase fold number
            fold_no = fold_no + 1

        precision_kfold = sum(precision_per_fold)/len(precision_per_fold)
        recall_kfold = sum(recall_per_fold)/len(recall_per_fold)
        fscore_kfold = sum(fscore_per_fold)/len(fscore_per_fold)
        mcc_kfold = sum(mcc_per_fold)/len(mcc_per_fold)

        print(f"C_VALUES = {C} ==> {precision_kfold}, {recall_kfold}, {fscore_kfold}, {mcc_kfold}")
        if fscore_kfold > best_performance[2]:
            best_performance[0] = precision_kfold
            best_performance[1] = recall_kfold
            best_performance[2] = fscore_kfold
            best_performance[3] = mcc_kfold

    return best_performance

In [ ]:
# Long Method
print(train_test_KFold(X_LM, y_LM))

============= RUNNING LOGISTIC REGRESSION =============
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
C_VALUES = 0.001 ==> 0.9712820512820514, 0.3071428571428571, 0.46185185185185185, 0.5180201759785211
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
C_VALUES = 0.01 ==> 0.8453701822122873, 0.587922077922078, 0.6930278628515455, 0.6750604614165099
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
C_VALUES = 0.1 ==> 0.823282647584973, 0.6386363636363637, 0.7148217713200667, 0.6930989641952555
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
C_VALUES = 1 ==> 0.8168319379033665, 0.6535064935064936, 0.7248178629880758, 0.6993903639553167
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
C_VALUES = 10 ==> 0.8162377575143532, 0.6498701298701299, 0.7231324529811926,

In [ ]:
# Feature Envy
print(train_test_KFold(X_FE, y_FE))

============= RUNNING LOGISTIC REGRESSION =============
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
C_VALUES = 0.001 ==> nan, 0.0, 0.0, -0.003270644018227125
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
C_VALUES = 0.01 ==> 0.36666666666666664, 0.045054945054945054, 0.07857142857142858, 0.1156677085816902
[0 1], [1741   52]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
C_VALUES = 0.1 ==> 0.36666666666666664, 0.06043956043956045, 0.09815347191508184, 0.12844806850670037
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
C_VALUES = 1 ==> 0.2, 0.07362637362637363, 0.10760233918128655, 0.10680011776309646
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[0 1], [1741   53]
[0 1], [1741   53]
C_VALUES = 10 ==> 0.1796825396825397, 0.07582417582417582, 0.10263347763347763, 0.09823739182324169
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
C_VALUES = 100 ==> 0.13571428571428573, 0.06153846153846154, 0.08352941176470588, 0.07246734162881123
[0.2, 0.07362637362637363, 0.10760233918128655, 0.10680011776309646]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
# God Class
print(train_test_KFold(X_GC, y_GC))

============= RUNNING LOGISTIC REGRESSION =============
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 0.001 ==> nan, 0.010526315789473684, 0.020455873758036234, 0.052157325168973215
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


C_VALUES = 0.01 ==> 0.7535714285714286, 0.06679197994987468, 0.12116563682887375, 0.19258079401077338
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 0.1 ==> 0.6997301930805767, 0.19743107769423557, 0.30529517164567976, 0.3277954029733788
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 1 ==> 0.569308371939951, 0.24649122807017543, 0.3425281506613064, 0.3175527517428128
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

C_VALUES = 10 ==> 0.6040986717267552, 0.2711152882205513, 0.3710240058852338, 0.34801400706933555
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 100 ==> 0.6231045751633987, 0.2744360902255639, 0.37596888344296187, 0.3570284060023101
[0.6231045751633987, 0.2744360902255639, 0.37596888344296187, 0.3570284060023101]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
# Data Class
print(train_test_KFold(X_DC, y_DC))

<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


============= RUNNING LOGISTIC REGRESSION =============
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 0.001 ==> nan, 0.010588972431077693, 0.02045587375803623, 0.044698055515760615
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 0.01 ==> 0.6957142857142856, 0.0668546365914787, 0.1216690589509204, 0.18755029349085017
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 0.1 ==> 0.6511455108359132, 0.1900375939849624, 0.2927679034791665, 0.30657309643533104
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 1 ==> 0.6247523310023311, 0.2644736842105263, 0.36743855558197375, 0.3522130439923616
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 10 ==> 0.5979891304347826, 0.28157894736842104, 0.38027452252608446, 0.35414465638671705
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
C_VALUES = 100 ==> 0.559109477124183, 0.27794486215538844, 0.3661361879616458, 0.331030796446815
[0.5979891304347826, 0.28157894736842104, 0.38027452252608446, 0.35414465638671705]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

# **CART**

In [ ]:
def train_test_KFold(X, y, feature_scaling=StandardScaler()):
    print("============= RUNNING CART =============")
    kfold = StratifiedKFold(n_splits=5, shuffle=True)

    # Define hyperparameters to search over
    max_depth_values = [None, 10, 20, 30]  # Example values for the maximum depth of the tree
    min_samples_split_values = [2, 5, 10]  # Example values for the minimum samples required to split a node
    min_samples_leaf_values = [1, 2, 4]  # Example values for the minimum samples required in a leaf node
    max_features_values = ['auto', 'sqrt', 'log2']  # Example values for the maximum number of features to consider for splitting

    best_performance = [0,0,0,0]

    # Iterate over var_smoothing values and choose the best one
    for max_depth in max_depth_values:
        for min_samples_split in min_samples_split_values:
            for min_samples_leaf in min_samples_leaf_values:
                for max_features in max_features_values:
                    fold_no = 1
                    precision_per_fold = []
                    recall_per_fold = []
                    fscore_per_fold = []
                    mcc_per_fold = []

                    classifier = DecisionTreeClassifier(
                        max_depth=max_depth,
                        min_samples_split=min_samples_split,
                        min_samples_leaf=min_samples_leaf,
                        max_features=max_features,
                        random_state=42
                    )

                    for train, test in kfold.split(X,y):
                        # print(f'Training for fold {fold_no}: ')
                        X_train = X[train]
                        X_test = X[test]
                        y_train = y[train]
                        y_test = y[test]
                        index, counts = np.unique(y_train, return_counts=True)
                        print(f'{index}, {counts}')

                        # Feature Scaling
                        if feature_scaling != None:
                            X_train = feature_scaling.fit_transform(X_train)
                            X_test = feature_scaling.transform(X_test)

                        # Define the model, fit and predict
                        classifier.fit(X_train, y_train)
                        y_pred = classifier.predict(X_test)

                        precision, recall, fscore, mcc = evaluation_metrics(y_test, y_pred)

                        precision_per_fold.append(precision)
                        recall_per_fold.append(recall)
                        fscore_per_fold.append(fscore)
                        mcc_per_fold.append(mcc)

                        # Increase fold number
                        fold_no = fold_no + 1

                    precision_kfold = sum(precision_per_fold)/len(precision_per_fold)
                    recall_kfold = sum(recall_per_fold)/len(recall_per_fold)
                    fscore_kfold = sum(fscore_per_fold)/len(fscore_per_fold)
                    mcc_kfold = sum(mcc_per_fold)/len(mcc_per_fold)

                    print(f"max_depth={max_depth} - min_samples_split={min_samples_split} - min_samples_leaf={min_samples_leaf} - max_features={max_features} ==> {precision_kfold}, {recall_kfold}, {fscore_kfold}, {mcc_kfold}")
                    if fscore_kfold > best_performance[2]:
                        best_performance[0] = precision_kfold
                        best_performance[1] = recall_kfold
                        best_performance[2] = fscore_kfold
                        best_performance[3] = mcc_kfold

    return best_performance

In [ ]:
# Long Method
print(train_test_KFold(X_LM, y_LM))

============= RUNNING CART =============
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=auto ==> 0.6586974185190726, 0.6427922077922078, 0.6504264090758628, 0.6056390823579105
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=sqrt ==> 0.6274168567925462, 0.667922077922078, 0.6453672358094419, 0.5985850475515703
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=log2 ==> 0.6426573426573425, 0.5848051948051948, 0.6112365384992106, 0.5650858029875004
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=auto ==> 0.7409948320413438, 0.6106493506493507, 0.6661935902115147, 0.6336843665845153
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=sqrt ==> 0.6858221818708141, 0.5667532467532468, 0.6201490351874757, 0.5796286722379803
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=log2 ==> 0.6727272921747315, 0.5994805194805195, 0.6326485285427967, 0.5901961921334535
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=auto ==> 0.7255549159218301, 0.585, 0.6469941780406401, 0.6113317833033964
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=sqrt ==> 0.7199991926433664, 0.5987012987012987, 0.651028105858825, 0.615546737312039
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=log2 ==> 0.7151380678316275, 0.6424025974025974, 0.6743596604101549, 0.6376217644063586
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=5 - min_samples_leaf=1 - max_features=auto ==> 0.6368824951188887, 0.5700000000000001, 0.5991803403568109, 0.553313375379943
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=None - min_samples_split=5 - min_samples_leaf=1 - max_features=sqrt ==> 0.6654794022293249, 0.6466883116883116, 0.6549072969984074, 0.6116826253185634
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221

...

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=auto ==> 0.6557407407407407, 0.6062337662337663, 0.6285682395217352, 0.5839184579367762
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=sqrt ==> 0.660053627452392, 0.6096753246753247, 0.6318750058192111, 0.5883383191755576
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=log2 ==> 0.6669413182978787, 0.6527272727272727, 0.6552728240051074, 0.6142624826417025
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=auto ==> 0.6897506597506597, 0.5484415584415584, 0.6051316435193579, 0.568613236736673
[

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=log2 ==> 0.7594946708333784, 0.6428571428571428, 0.6943167909461236, 0.6624846529562051
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=auto ==> 0.7009560345877481, 0.5738311688311688, 0.6290665628132085, 0.5910382819464337
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=sqrt ==> 0.6732543503008619, 0.5807792207792208, 0.6216309112549714, 0.5797723187346564
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=log2 ==> 0.7024491978609625, 0.6064285714285715, 0.6488897605100261, 0.6103879302508519
[0.759494670833378

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

In [ ]:
# Feature Envy
print(train_test_KFold(X_FE, y_FE))

============= RUNNING CART =============
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=auto ==> 0.11939393939393939, 0.10659340659340662, 0.11154481410321233, 0.08584526881137076
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=sqrt ==> 0.15300751879699245, 0.16703296703296705, 0.15834330484330483, 0.1329081442773853
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=log2 ==> 0.12156288156288157, 0.13626373626373628, 0.12800425381070543, 0.10027597103489785
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=auto ==> 0.38730158730158726, 0.1527472527472528, 0.18392496392496396, 0.19739049625094132
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=sqrt ==> 0.16818181818181815, 0.13516483516483516, 0.14944927536231883, 0.13054698110268906
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=log2 ==> 0.03333333333333333, 0.014285714285714285, 0.02, 0.0024872614638534207
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=auto ==> nan, 0.09010989010989011, 0.11097308488612836, 0.09842789235042797
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
<ipython-input-30-ec

[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=sqrt ==> 0.26666666666666666, 0.13516483516483518, 0.17482837528604117, 0.16925059958243632
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=log2 ==> 0.24666666666666667, 0.09230769230769231, 0.13260233918128655, 0.13514767630522262
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=5 - min_samples_leaf=1 - max_features=auto ==> 0.16772430184194892, 0.13626373626373628, 0.1486335403726708, 0.12534351745273045
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=None - min_samples_split=5 - min_samples_leaf=1 - max_features=sqrt ==> 0.06352941176470588, 0.06153846153846154, 0.06144927536231883, 0.04217417733977309
[0 1], [1741   52]
[0 1], [1740   53]

...

[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=5 - min_samples_leaf=4 - max_features=auto ==> 0.152014652014652, 0.10659340659340662, 0.12163871216502795, 0.10649732669041571
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=5 - min_samples_leaf=4 - max_features=sqrt ==> 0.25357142857142856, 0.10659340659340662, 0.14686529157117392, 0.1443237715135073
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=5 - min_samples_leaf=4 - max_features=log2 ==> nan, 0.05934065934065934, 0.08468229397021966, 0.08217394685425891
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=auto ==> 0.20949197860962565, 0.15384615384615385, 0.17074410163339385, 0.15389804031196108
[0 1], [1741   52]
[0 1], [1740   53]
[0 1]

<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will 

[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=sqrt ==> 0.3715909090909091, 0.18241758241758244, 0.19661740558292284, 0.2039644050865157
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=log2 ==> 0.16843156843156842, 0.1340659340659341, 0.14810256410256412, 0.12829404185999624
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=auto ==> 0.15333333333333332, 0.06153846153846154, 0.08550275945618521, 0.07903409065237657
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=sqrt ==> 0.15388888888888888, 0.089010989010989, 0.11162557249513771, 0.09848772929555126
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], 

/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=log2 ==> 0.11499999999999999, 0.09120879120879122, 0.09913419913419913, 0.08047259796058542
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=auto ==> 0.19666666666666668, 0.13626373626373628, 0.15756521739130433, 0.14374636821871625
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=sqrt ==> 0.20428571428571426, 0.09010989010989011, 0.1238468720821662, 0.11800594701739335
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=log2 ==> 0.29725274725274725, 0.14945054945054945, 0.1898474945533769, 0.18808176246059974
[0.46111111111111114, 0

In [ ]:
# God Class
print(train_test_KFold(X_GC, y_GC))

============= RUNNING CART =============
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=auto ==> 0.4628136562526322, 0.4716165413533834, 0.4656082415817977, 0.3850382208046318
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=sqrt ==> 0.487245945378993, 0.468295739348371, 0.47630345388966083, 0.39907034054159435
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=log2 ==> 0.45581584210068693, 0.44342105263157894, 0.44894712202517717, 0.3671827534070033
[0 1], [1500  227]
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=auto ==> 0.43826188336674277, 0.3554511278195489, 0.39129145168736895, 0.31336747105421364
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=sqrt ==> 0.51455734378097, 0.418984962406015, 0.45940723987505716, 0.39018173038302123
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=log2 ==> 0.4769672131147541, 0.39786967418546365, 0.43047905125830865, 0.35715300311926274
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=auto ==> 0.5154179385727493, 0.4152255639097745, 0.4580133555963541, 0.3903739610571003
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=sqrt ==> 0.5105967400041368, 0.3943609022556391, 0.4404377527911122, 0.376155659378988
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=log2 ==> 0.5054839268548947, 0.41873433583959896, 0.45416562837031826, 0.38523052807150465
[0 1], [1500  227]
[0 1], [1500  227]


...

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=log2 ==> 0.45941743553144604, 0.42944862155388475, 0.44230670968375885, 0.3623298218783641
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=auto ==> 0.5014279301807434, 0.41578947368421054, 0.4513701085194265, 0.38211543419014155
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=sqrt ==> 0.4698709281722775, 0.3905388471177945, 0.4249135017126954, 0.34965078357168156
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=log2 ==> 0.4853407850602478, 0.40513784461152885, 0.4342056053863283, 0.3653733463502496
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=auto ==> 0.4479849465998461, 0.39097744360902253, 0.4154498433722866, 0.3358765252423049
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=sqrt ==> 0.4788441744590813, 0.4013784461152882, 0.43602837398049044, 0.3612197376603972
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=log2 ==> 0.4964569697997672, 0.37337092731829574, 0.4228062462143036, 0.3555037511483439
[0.5296340981557297, 0.45050125313283207, 0.48447052356029996, 0.41659941431804404]


In [ ]:
# Data Class
print(train_test_KFold(X_DC, y_DC))

============= RUNNING CART =============
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=auto ==> 0.3826318033214585, 0.42932330827067666, 0.4035664905675348, 0.30971845042838714
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=sqrt ==> 0.4301343402331922, 0.47531328320802013, 0.4496085597631576, 0.36205104531461807


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=1 - max_features=log2 ==> 0.42294400557176137, 0.4507518796992481, 0.43498391844136675, 0.3486052872098501
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=auto ==> 0.48487902237902236, 0.37669172932330824, 0.4221052305442276, 0.3516653639814452
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=sqrt ==> 0.5015316325577049, 0.4119047619047619, 0.4479328271425609, 0.37748925630388536
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=2 - max_features=log2 ==> 0.48803921568627445, 0.3905388471177945, 0.4329642093284063, 0.36162850897628634
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=None - min_samples_split=2 - min_samples_leaf=4 - max_features=auto ==> 0.43696000585841466, 0.3625939849624061, 0.3938282939921033, 0.31480636951655033
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]


...

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=auto ==> 0.46433708567854903, 0.39404761904761904, 0.42286851042597196, 0.3481088964617498
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=sqrt ==> 0.491113653699466, 0.41535087719298247, 0.4481869966895357, 0.3749609475168726
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=1 - max_features=log2 ==> 0.4914385435992579, 0.45093984962406014, 0.4681343987117025, 0.3924821474688877
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=auto ==> 0.4878795930552095, 0.362280701754386, 0.4096320734843578, 0.34398484600262147
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=sqrt ==> 0.5524670255249177, 0.42625313283208016, 0.4807956794972979, 0.4184216417067237
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=2 - max_features=log2 ==> 0.5449910031488978, 0.41885964912280704, 0.4708452230754453, 0.40943497547643126
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=auto ==> 0.5221021021021021, 0.38734335839598993, 0.4441297661681062, 0.37934951036524844
[0 1], [1500  227]


/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py:269: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
/usr/local/lib/pytho

[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=sqrt ==> 0.5193317793317794, 0.3872807017543859, 0.44330056619973696, 0.37819186584715686
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
max_depth=30 - min_samples_split=10 - min_samples_leaf=4 - max_features=log2 ==> 0.5093313069908814, 0.4087719298245614, 0.44814404525831997, 0.38223458756529954
[0.5524670255249177, 0.42625313283208016, 0.4807956794972979, 0.4184216417067237]


# **SVM**

In [ ]:
from sklearn.svm import SVC

def train_test_KFold(X, y, feature_scaling=StandardScaler()):
    print("============= RUNNING SVM =============")
    kfold = StratifiedKFold(n_splits=5, shuffle=True)

    # Define hyperparameters to search over
    gamma_values = [0.001, 0.01, 0.1, 1, 10, 100]  # Example values for the gamma parameter
    C_values = [0.001, 0.01, 0.1, 1, 10, 100]  # Example values for the C parameter

    best_performance = [0,0,0,0]

    # Iterate over var_smoothing values and choose the best one
    for gamma in gamma_values:
        for C in C_values:
            fold_no = 1
            precision_per_fold = []
            recall_per_fold = []
            fscore_per_fold = []
            mcc_per_fold = []

            classifier = SVC(kernel='rbf', gamma=gamma, C=C, random_state=42)
            for train, test in kfold.split(X,y):
                # print(f'Training for fold {fold_no}: ')
                X_train = X[train]
                X_test = X[test]
                y_train = y[train]
                y_test = y[test]
                index, counts = np.unique(y_train, return_counts=True)
                print(f'{index}, {counts}')

                # Feature Scaling
                if feature_scaling != None:
                    X_train = feature_scaling.fit_transform(X_train)
                    X_test = feature_scaling.transform(X_test)

                # Define the model, fit and predict
                classifier.fit(X_train, y_train)
                y_pred = classifier.predict(X_test)

                precision, recall, fscore, mcc = evaluation_metrics(y_test, y_pred)

                precision_per_fold.append(precision)
                recall_per_fold.append(recall)
                fscore_per_fold.append(fscore)
                mcc_per_fold.append(mcc)

                # Increase fold number
                fold_no = fold_no + 1

            precision_kfold = sum(precision_per_fold)/len(precision_per_fold)
            recall_kfold = sum(recall_per_fold)/len(recall_per_fold)
            fscore_kfold = sum(fscore_per_fold)/len(fscore_per_fold)
            mcc_kfold = sum(mcc_per_fold)/len(mcc_per_fold)

            print(f"Kernerl func='rbf' - Gamma={gamma} - C={C} ==> {precision_kfold}, {recall_kfold}, {fscore_kfold}, {mcc_kfold}")
            if fscore_kfold > best_performance[2]:
                best_performance[0] = precision_kfold
                best_performance[1] = recall_kfold
                best_performance[2] = fscore_kfold
                best_performance[3] = mcc_kfold

    return best_performance

In [ ]:
# Long Method
print(train_test_KFold(X_LM, y_LM))

============= RUNNING SVM =============
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1705  222]
[0 1], [1705  222]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Kernerl func='rbf' - Gamma=0.001 - C=0.001 ==> nan, 0.0, 0.0, 0.0
[0 1], [1704  222]
[0 1], [1705  221]


...

<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1705  222]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1705  222]
Kernerl func='rbf' - Gamma=100 - C=100 ==> nan, 0.0, 0.0, 0.0
[0.7559987042992116, 0.6901298701298701, 0.7198610045051451, 0.6874124993252101]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


In [ ]:
# Feature Envy
print(train_test_KFold(X_FE, y_FE))

<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


============= RUNNING SVM =============
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]
Kernerl func='rbf' - Gamma=0.001 - C=0.001 ==> nan, 0.0, 0.0, 0.0
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1741   53]
[0 1], [1741   53]
Kernerl func='rbf' - Gamma=0.001 - C=0.01 ==> nan, 0.0, 0.0, 0.0
[0 1], [1741   52]
[0 1], [1740   53]
[0 1], [1741   53]
[0 1], [1741   53]
[0 1], [1741   53]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


...

<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1741   53]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1741   53]
Kernerl func='rbf' - Gamma=100 - C=100 ==> nan, 0.0, 0.0, 0.0
[0.2246031746031746, 0.12197802197802199, 0.15449927849927853, 0.14410839954310947]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


In [ ]:
# God Class
print(train_test_KFold(X_GC, y_GC))

<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


============= RUNNING SVM =============
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1500  227]
[0 1], [1500  228]
Kernerl func='rbf' - Gamma=0.001 - C=0.001 ==> nan, 0.0, 0.0, 0.0
[0 1], [1500  227]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


...

[0 1], [1500  228]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Kernerl func='rbf' - Gamma=100 - C=0.1 ==> nan, 0.0, 0.0, 0.0
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Kernerl func='rbf' - Gamma=100 - C=1 ==> nan, 0.02807017543859649, 0.05302047503616906, 0.11016347196959782
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
Kernerl func='rbf' - Gamma=100 - C=10 ==> 0.5533333333333333, 0.03176691729323308, 0.059679540613133276, 0.10741319862590833
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
Kernerl func='rbf' - Gamma=100 - C=100 ==> 0.6666666666666666, 0.03527568922305764, 0.06690006483282393, 0.1320420018192159
[0.61091542959964, 0.4010651629072681, 0.4809105089040102, 0.4339047850810148]


In [ ]:
# Data Class
print(train_test_KFold(X_DC, y_DC))

============= RUNNING SVM =============
[0 1], [1500  227]
[0 1], [1500  227]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1500  227]
[0 1], [1500  227]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)
<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1500  228]


...

[0 1], [1500  227]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


[0 1], [1500  228]


<ipython-input-30-ec84681f9a3e>:6: RuntimeWarning: invalid value encountered in long_scalars
  precision = tp / (tp + fp)


Kernerl func='rbf' - Gamma=100 - C=0.1 ==> nan, 0.0, 0.0, 0.0
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
Kernerl func='rbf' - Gamma=100 - C=1 ==> 0.45999999999999996, 0.03182957393483709, 0.05934426229508196, 0.09959526153465613
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
Kernerl func='rbf' - Gamma=100 - C=10 ==> 0.5166666666666667, 0.031578947368421054, 0.059453551912568306, 0.10391074545960595
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  227]
[0 1], [1500  228]
Kernerl func='rbf' - Gamma=100 - C=100 ==> 0.5966666666666667, 0.03176691729323308, 0.05990887552396573, 0.11261250823878806
[0.6342914363370696, 0.3837719298245614, 0.4771765157632978, 0.4365850503738236]


# **BP**

In [ ]:
from sklearn.neural_network import MLPClassifier

def train_test_KFold(X, y, feature_scaling=StandardScaler()):
    print("============= RUNNING Back propagation NN model =============")
    kfold = StratifiedKFold(n_splits=5, shuffle=True)

    # Define hyperparameters to search over
    layer_sizes = [(16,), (32,), (64,), (32, 16)]
    learning_rates = [0.001, 0.01, 0.1]
    max_iterations = [100, 200, 300]
    tolerant_error = [1e-4, 1e-3, 1e-2]

    best_performance = [0,0,0,0]

    # Iterate over var_smoothing values and choose the best one
    for layers in layer_sizes:
        for learning_rate in learning_rates:
            for max_iter in max_iterations:
                for tol_err in tolerant_error:
                    fold_no = 1
                    precision_per_fold = []
                    recall_per_fold = []
                    fscore_per_fold = []
                    mcc_per_fold = []

                    classifier = MLPClassifier(hidden_layer_sizes=layers,
                                    learning_rate_init=learning_rate,
                                    max_iter=max_iter,
                                    tol=tol_err,
                                    random_state=42)
                    for train, test in kfold.split(X,y):
                        # print(f'Training for fold {fold_no}: ')
                        X_train = X[train]
                        X_test = X[test]
                        y_train = y[train]
                        y_test = y[test]
                        index, counts = np.unique(y_train, return_counts=True)
                        print(f'{index}, {counts}')

                        # Feature Scaling
                        if feature_scaling != None:
                            X_train = feature_scaling.fit_transform(X_train)
                            X_test = feature_scaling.transform(X_test)

                        # Define the model, fit and predict
                        classifier.fit(X_train, y_train)
                        y_pred = classifier.predict(X_test)

                        precision, recall, fscore, mcc = evaluation_metrics(y_test, y_pred)

                        precision_per_fold.append(precision)
                        recall_per_fold.append(recall)
                        fscore_per_fold.append(fscore)
                        mcc_per_fold.append(mcc)

                        # Increase fold number
                        fold_no = fold_no + 1

                    precision_kfold = sum(precision_per_fold)/len(precision_per_fold)
                    recall_kfold = sum(recall_per_fold)/len(recall_per_fold)
                    fscore_kfold = sum(fscore_per_fold)/len(fscore_per_fold)
                    mcc_kfold = sum(mcc_per_fold)/len(mcc_per_fold)

                    print(f"layers={layers} - learning_rate={learning_rate} - max_iter={max_iter} - tol_err={tol_err}==> {precision_kfold}, {recall_kfold}, {fscore_kfold}, {mcc_kfold}")
                    if fscore_kfold > best_performance[2]:
                        best_performance[0] = precision_kfold
                        best_performance[1] = recall_kfold
                        best_performance[2] = fscore_kfold
                        best_performance[3] = mcc_kfold

    return best_performance

In [ ]:
# Long Method
print(train_test_KFold(X_LM, y_LM))

============= RUNNING Back propagation NN model =============
[0 1], [1704  222]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[0 1], [1705  221]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


[0 1], [1705  221]


...

[0 1], [1705  222]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[0 1], [1705  222]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


layers=(32, 16) - learning_rate=0.001 - max_iter=300 - tol_err=0.0001==> 0.7119999256533214, 0.6462987012987013, 0.6742490120716722, 0.6372952735034707
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
layers=(32, 16) - learning_rate=0.001 - max_iter=300 - tol_err=0.001==> 0.7964841396420344, 0.6533116883116883, 0.7132832789112816, 0.6872312582838365
[0 1], [1704  222]
[0 1], [1705  221]
[0 1], [1705  221]
[0 1], [1705  222]
[0 1], [1705  222]
layers=(32, 16) - learning_rate=0.001 - max_iter=300 - tol_err=0.01==> 0.8021962566048557, 0.650064935064935, 0.7174335681933359, 0.6903613134188012
[0 1], [1704  222]


In [ ]:
# Feature Envy
print(train_test_KFold(X_FE, y_FE))

In [ ]:
# God Class
print(train_test_KFold(X_GC, y_GC))

In [ ]:
# Data Class
print(train_test_KFold(X_DC, y_DC))